<a href="https://colab.research.google.com/github/MannoMation/SkystoneSimulator/blob/master/GoalDetection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##### Copyright 2020 The TensorFlow Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Flower classification with TensorFlow Lite Model Maker with TensorFlow 2.0

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/tensorflow/examples/blob/master/lite/codelabs/flower_classification/ml/Flower_Classification_with_TFLite_Model_Maker.ipynb">      
    <img src="https://www.tensorflow.org/images/colab_logo_32px.png" />
    Run in Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/tensorflow/examples/blob/master/lite/codelabs/flower_classification/ml/Flower_Classification_with_TFLite_Model_Maker.ipynb">
    <img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />
    View source on GitHub</a>
  </td>
</table>

Model Maker library simplifies the process of adapting and converting a TensorFlow neural-network model to particular input data when deploying this model for on-device ML applications.

This notebook shows an end-to-end example that utilizes this Model Maker library to illustrate the adaption and conversion of a commonly-used image classification model to classify flowers on a mobile device.

## Prerequisites

To run this example, we first need to make a copy of this notebook. Click on "Copy to Drive" at the top of this notebook. Then we need to install serveral required packages, including Model Maker package that in github [repo](https://github.com/tensorflow/examples/tree/master/tensorflow_examples/lite/model_maker).

In [3]:
!pip install -q tflite-model-maker

     |████████████████████████████████| 593kB 27.8MB/s 
     |████████████████████████████████| 1.1MB 53.5MB/s 
     |████████████████████████████████| 1.2MB 48.2MB/s 
     |████████████████████████████████| 645kB 49.5MB/s 
     |████████████████████████████████| 215kB 56.1MB/s 
     |████████████████████████████████| 6.3MB 48.6MB/s 
     |████████████████████████████████| 112kB 58.9MB/s 
     |████████████████████████████████| 122kB 62.4MB/s 
     |████████████████████████████████| 849kB 52.8MB/s 
     |████████████████████████████████| 686kB 42.0MB/s 
     |████████████████████████████████| 71kB 11.5MB/s 
     |████████████████████████████████| 92kB 13.7MB/s 
     |████████████████████████████████| 204kB 55.6MB/s 
     |████████████████████████████████| 37.2MB 69kB/s 
     |████████████████████████████████| 102kB 14.7MB/s 
     |████████████████████████████████| 358kB 47.4MB/s 


In [4]:
pip install -q pycocotools

Import the required packages.

In [19]:
from google.colab import drive
drive.mount('/gdrive')
%cd /gdrive

from tflite_model_maker import image_classifier
from tflite_model_maker.image_classifier import DataLoader
from tflite_model_maker import object_detector
from tflite_model_maker import model_spec

import tensorflow as tf
assert tf.__version__.startswith('2')

tf.get_logger().setLevel('ERROR')
from absl import logging
logging.set_verbosity(logging.ERROR)

import matplotlib.pyplot as plt
import numpy as np

Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).
/gdrive


## Simple End-to-End Example

In [6]:
train_data = object_detector.DataLoader.from_pascal_voc("/gdrive/MyDrive/Goal Detection/TrainingData", "/gdrive/MyDrive/Goal Detection/TrainingData", label_map={1: "Goal"})
print(len(train_data))
#validation_data = object_detector.DataLoader.from_pascal_voc("/gdrive/MyDrive/Goal Detection/TrainingData/", "/gdrive/MyDrive/Goal Detection/TrainingData/", label_map={1: "Goal"})
#test_data = object_detector.DataLoader.from_pascal_voc("/gdrive/MyDrive/Goal Detection/TrainingData/", "/gdrive/MyDrive/Goal Detection/TrainingData/", label_map={1: "Goal"})

#train_data, test_data = dataloader.split(0.9)
#train_data = dataloader.gen_dataset
#test_data = dataloader.gen_dataset

INFO:tensorflow:Cache will be stored in /tmp/tmpg_g1nlk3 with prefix filename cf0923bf2349a985988c77584edda788. Cache_prefix is /tmp/tmpg_g1nlk3/cf0923bf2349a985988c77584edda788
INFO:tensorflow:On image 0
83


In [8]:
validation_data = train_data
test_data = train_data

### Get the data path

Let's get some images to play with this simple end-to-end example. Hundreds of images is a good start for Model Maker while more data could achieve better accuracy.

In [ ]:
image_path = tf.keras.utils.get_file(
      'flower_photos',
      'https://storage.googleapis.com/download.tensorflow.org/example_images/flower_photos.tgz',
      untar=True)

228818944/228813984 [==============================] - 1s 0us/step


You could replace `image_path` with your own image folders. As for uploading data to colab, you could find the upload button in the left sidebar shown in the image below with the red rectangle. Just have a try to upload a zip file and unzip it. The root file path is the current path.

<img src="https://storage.googleapis.com/download.tensorflow.org/models/tflite/screenshots/model_maker_image_classification.png" alt="Upload File" width="800" hspace="100">

If you prefer not to upload your images to the cloud, you could try to run the library locally following the [guide](https://github.com/tensorflow/examples/tree/master/tensorflow_examples/lite/model_maker) in github.

### Run the example
The example just consists of 4 lines of code as shown below, each of which representing one step of the overall process.


1.   Load input data specific to an on-device ML app. Split it to training data and testing data.

In [ ]:
data = DataLoader.from_folder(image_path)
train_data, test_data = data.split(0.9)

INFO:tensorflow:Load image with size: 3670, num_label: 5, labels: daisy, dandelion, roses, sunflowers, tulips.


2. Customize the TensorFlow model.

In [16]:
#model = image_classifier.create(train_data)
spec = model_spec.get('efficientdet_lite0')
model = object_detector.create(train_data, model_spec=spec, epochs=50, batch_size=8, train_whole_model=True, validation_data=validation_data)

INFO:tensorflow:Retraining the models...


INFO:tensorflow:Retraining the models...


Epoch 1/50
 6/10 [=================>............] - ETA: 1s - det_loss: 1.7106 - cls_loss: 1.1382 - box_loss: 0.0114 - reg_l2_loss: 0.0630 - loss: 1.7736 - learning_rate: 0.0085 - gradient_norm: 2.1092WARNING:tensorflow:Callback method `on_train_batch_end` is slow compared to the batch time (batch time: 0.1911s vs `on_train_batch_end` time: 0.2355s). Check your callbacks.


10/10 [==============================] - 33s 701ms/step - det_loss: 1.6640 - cls_loss: 1.1070 - box_loss: 0.0111 - reg_l2_loss: 0.0630 - loss: 1.7270 - learning_rate: 0.0090 - gradient_norm: 1.9721 - val_det_loss: 1.5032 - val_cls_loss: 1.0304 - val_box_loss: 0.0095 - val_reg_l2_loss: 0.0630 - val_loss: 1.5662
Epoch 2/50
10/10 [==============================] - 3s 315ms/step - det_loss: 1.2850 - cls_loss: 0.8146 - box_loss: 0.0094 - reg_l2_loss: 0.0630 - loss: 1.3480 - learning_rate: 0.0100 - gradient_norm: 2.1927 - val_det_loss: 1.6472 - val_cls_loss: 1.2011 - val_box_loss: 0.0089 - val_reg_l2_loss: 0.0630 - val_loss: 1.7102
Epoch 3/50
10/10 [==============================] - 3s 314ms/step - det_loss: 0.8739 - cls_loss: 0.5021 - box_loss: 0.0074 - reg_l2_loss: 0.0630 - loss: 0.9369 - learning_rate: 0.0099 - gradient_norm: 2.9497 - val_det_loss: 0.9252 - val_cls_loss: 0.5417 - val_box_loss: 0.0077 - val_reg_l2_loss: 0.0630 - val_loss: 0.9882
Epoch 4/50
10/10 [==========================

3. Evaluate the model.

In [17]:
model.evaluate(test_data)

1/2 [==============>...............] - ETA: 6sWARNING:tensorflow:6 out of the last 6 calls to <function EfficientDetModelSpec.evaluate.<locals>._get_detections at 0x7f4895b7af80> triggered tf.function retracing. Tracing is expensive and the excessive number of tracings could be due to (1) creating @tf.function repeatedly in a loop, (2) passing tensors with different shapes, (3) passing Python objects instead of tensors. For (1), please define your @tf.function outside of the loop. For (2), @tf.function has experimental_relax_shapes=True option that relaxes argument shapes that can avoid unnecessary retracing. For (3), please refer to https://www.tensorflow.org/guide/function#controlling_retracing and https://www.tensorflow.org/api_docs/python/tf/function for  more details.


2/2 [==============================] - 9s 3s/step

loading annotations into memory...
Done (t=0.00s)
creating index...
index created!
Loading and preparing results...
Converting ndarray to lists...
(8300, 7)
0/8300
DONE (t=0.03s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=0.33s).
Accumulating evaluation results...
DONE (t=0.04s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.878
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 1.000
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 1.000
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = -1.000
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.886
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = 0.878
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.888
 Average Recall     (AR) @[ IoU=0.50:0.95 | a

{'AP': 0.8778713,
 'AP50': 1.0,
 'AP75': 1.0,
 'AP_/Goal': 0.8778713,
 'APl': 0.8777109,
 'APm': 0.8859736,
 'APs': -1.0,
 'ARl': 0.9116883,
 'ARm': 0.9,
 'ARmax1': 0.8879518,
 'ARmax10': 0.9108434,
 'ARmax100': 0.9108434,
 'ARs': -1.0}

4.  Export to TensorFlow Lite model.
You could download it in the left sidebar same as the uploading part for your own use.

In [20]:
#model.export(export_dir='.')
model.export(export_dir='.')

UnimplementedError: ignored

5. Download the trained model by clicking on the folder icon on the left hand side. Right-click on "model.tflite" and select download. Or run the following code:

In [ ]:
from google.colab import files
files.download('model.tflite') 

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

After this simple 5 steps, we can now continue to the next step in the [codelab](https://codelabs.developers.google.com/codelabs/recognize-flowers-with-tensorflow-on-android-beta/#2).

For a more comprehensive guide to TFLite Model Maker, please refer to this [notebook](https://colab.sandbox.google.com/github/tensorflow/examples/blob/master/tensorflow_examples/lite/model_maker/demo/image_classification.ipynb) and its [documentation](https://github.com/tensorflow/examples/tree/master/tensorflow_examples/lite/model_maker).